In [1]:
!pip install gluonts typer accelerate typer_config triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.0 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/erfan-bayat13/chronos-forecasting.git

Cloning into 'chronos-forecasting'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 353 (delta 137), reused 141 (delta 77), pack-reused 119 (from 1)
Receiving objects: 100% (353/353), 964.40 KiB | 11.21 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [3]:
import requests
import os
import pandas as pd
from tqdm import tqdm

# Create a data directory
!mkdir -p data

# Function to download file with progress bar
def download_file(url, filename):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    with open(filename, 'wb') as f, tqdm(
        total=total_size,
        unit='iB',
        unit_scale=True
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

In [4]:
# Solar (Hourly)
!wget https://www.nrel.gov/grid/solar-power-data.html -O data/solar_hourly.csv

# Wind Farms (Hourly) - from Monash Time Series Repository
!wget https://zenodo.org/record/4656069/files/wind_farms_minutely.zip
!unzip wind_farms_minutely.zip -d data/

--2024-11-30 17:05:56--  https://www.nrel.gov/grid/solar-power-data.html
Resolving www.nrel.gov (www.nrel.gov)... 192.174.63.97, 2620:d0:8000:7::97
Connecting to www.nrel.gov (www.nrel.gov)|192.174.63.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/solar_hourly.csv’

data/solar_hourly.c     [   <=>              ]  53.41K   131KB/s    in 0.4s    

2024-11-30 17:05:57 (131 KB/s) - ‘data/solar_hourly.csv’ saved [54695]

--2024-11-30 17:05:57--  https://zenodo.org/record/4656069/files/wind_farms_minutely.zip
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4656069/files/wind_farms_minutely.zip [following]
--2024-11-30 17:05:58--  https://zenodo.org/records/4656069/files/wind_farms_minutely.zip
Reusing existing connection to zenodo.or

In [5]:
!wget https://zenodo.org/records/4656140/files/electricity_hourly_dataset.zip?download=1
!unzip /content/electricity_hourly_dataset.zip?download=1 -d data/

--2024-11-30 17:05:58--  https://zenodo.org/records/4656140/files/electricity_hourly_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11823931 (11M) [application/octet-stream]
Saving to: ‘electricity_hourly_dataset.zip?download=1’

electricity_hourly_ 100%[===================>]  11.28M  5.17MB/s    in 2.2s    

2024-11-30 17:06:01 (5.17 MB/s) - ‘electricity_hourly_dataset.zip?download=1’ saved [11823931/11823931]



In [7]:
import pandas as pd
from gluonts.dataset.common import ListDataset
def print_header(file, n_rows=9):
    df_first_rows = pd.read_csv(file, sep='\t', skiprows = range(n_rows, 338))
    print("---------- Dataset information ----------\n")
    for i in range(n_rows-1):
        print(df_first_rows.iloc[i,0])
    print('\n')

print_header('data/electricity_hourly_dataset.tsf')
df = pd.read_csv('data/electricity_hourly_dataset.tsf', sep=',', skiprows = range(17), header=None)
starting_date = df.iloc[0,0].split(':')[1]
values = []
for date in df.iloc[:,0]:
    tmp_dates = date.split(':')
    values.append(int(tmp_dates[2]))
df.iloc[:,0] = values
print('Starting date:', starting_date, end='\n\n\n')
display(df)
list_for_gluon = []
for row in df.values:
    gluon_header = {"start": starting_date, "target": row}
    list_for_gluon.append(gluon_header)
dataset = ListDataset(list_for_gluon,freq="h")


---------- Dataset information ----------

# The electricity dataset represents the electricity consumption of 370 clients recorded in 15-minutes periods in Kilowatt (kW) from 2011 to 2014.
# This file contains an aggregated version of the original dataset used by Lai et al. (2017).
# This contains 321 hourly time series from 2012 to 2014.
#
# For more details, please refer to
# UCI, 2020. Electricityloaddiagrams20112014 data set, uci machine learning repository. Accessed:  2020-04-30. URL https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014
# Lai, G., 2017. multivariate-time-series-data. https://github.com/laiguokun/multivariate-time-series-data, accessed:  2020-05-04
# Lai, G., Chang, W., Yang, Y., Liu, H., 2017. Modeling long and short-term temporal patterns with deep neural networks. CoRR abs/1703.07015.URL http://arxiv.org/abs/1703.07015


Starting date: 2012-01-01 00-00-01




,0,1,2,3,4,5,6,7,8,9,...,26294,26295,26296,26297,26298,26299,26300,26301,26302,26303
0,14,18,21,20,22,20,20,20,13,11,...,8,17,13,10,11,11,11,12,10,11
1,69,92,96,92,91,92,91,92,91,96,...,109,117,118,115,117,116,103,93,92,88
2,234,312,312,312,312,187,138,8,7,8,...,7,7,8,10,12,8,8,8,8,8
3,415,556,560,443,346,340,376,361,259,402,...,550,564,620,669,830,844,749,650,646,648
4,215,292,272,213,190,178,199,187,144,172,...,259,245,267,274,351,384,371,346,349,337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,48,65,64,65,75,64,65,69,77,80,...,478,480,513,514,525,324,302,302,259,115
317,38,47,43,39,40,39,49,55,40,38,...,16,20,26,39,26,21,20,18,33,31
318,1558,2177,2193,1315,1378,1250,1818,2017,1261,1228,...,2194,2030,2055,1989,2140,1870,1506,1864,2623,2706
319,182,253,218,195,191,185,191,192,204,190,...,656,627,487,467,226,162,438,621,783,647


In [8]:
# Save function
import os
import json
def save_dataset(dataset, path):
    os.makedirs(path, exist_ok=True)
    with open(os.path.join(path, "data.json"), "w") as f:
        for entry in dataset:
            entry = {
                **entry,
                "start": str(entry["start"]),
                "target": entry["target"].tolist()
            }
            json.dump(entry, f)
            f.write("\n")

# Save electricity dataset
save_dataset(dataset,'data/elec_hourly')

In [9]:
!mkdir data/configs
!cp chronos-forecasting/scripts/training/configs/chronos-t5-mini.yaml data/configs/bob.yaml

In [10]:
!cp /content/data/configs/bob.yaml data/configs/bob2.yaml

In [ ]:
!pip install chronos-forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


In [13]:
!pip install -e /content/chronos-forecasting

Obtaining file:///content/chronos-forecasting
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 24.4 MB/s eta 0:00:00
  Building editable for chronos-forecasting (pyproject.toml) ... done
  Created wheel for chronos-forecasting: filename=chronos_forecasting-1.3.0-py3-none-any.whl size=14956 sha256=c6dbfcf17a949036d9ea3ef1800403264470936c42d540086e9700e99bbe33b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-qx215tx_/wheels/ba/d3/d9/895a35f5c2d461dabbd8d388a3dd3c35f4f42bc8be53e1f22c
Successfully built chronos-forecasting
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


In [14]:
!python3 chronos-forecasting/scripts/training/train.py --config data/configs/bob.yaml

2024-11-30 17:07:53.915694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 17:07:53.948789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 17:07:53.961246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 17:07:53.987768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 17:07:55.503361: W tensorflow/comp